## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars-dev.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [4]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                sleep(400)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=72418a31-e5fb-47e2-ad89-b7b96e35db55
Done
Done
ara-aragorn Done 280
Done
ara-arax Done 138
Error
ara-bte Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2b_RHOBTB2_twohop.json
A.2b_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=10d8a126-2785-4695-98f5-650f1a85fc85
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=30daf606-6f8f-4d82-bd0f-9f21bd017552
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=3ab1c5de-810f-4b9b-88d0-dec261b1b6d6
Done
Done
ara-aragorn Done 1017
Done
ara-ara

Done
Done
ara-arax ARS Error 0
Done
ara-aragorn Done 0
Error
ara-bte Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.3d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin.json
B.3d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin
https://arax.ncats.io/?source=ARS&id=cdeba98a-d83a-4f6c-83dd-094417c2454d
Done
Done
ara-arax Done 0
Done
ara-aragorn Done 0
Error
ara-bte Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI.json
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI
https://arax.ncats.io/?source=ARS&id=341a05db-235c-451e-b43a-b2553ae51a49
Done
Done
ara-aragorn Done 0
Done
ara-arax ARS Error 0
Error
ara-bte Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease.json
B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
https://arax.ncats.io/?source=ARS&id=c64d0d7e-e173-4b9f-a3ad-3f92e6ef6ca9

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [5]:
dict_workflows

{'A.3_KCNMA1': '72418a31-e5fb-47e2-ad89-b7b96e35db55',
 'A.2b_RHOBTB2_twohop': '10d8a126-2785-4695-98f5-650f1a85fc85',
 'A.2a_RHOBTB2_twohop': '30daf606-6f8f-4d82-bd0f-9f21bd017552',
 'A.8_EGFR_simple': '3ab1c5de-810f-4b9b-88d0-dec261b1b6d6',
 'A.0_RHOBTB2_direct': '05aec4a6-9dee-4578-aaad-58858215300e',
 'A.2_RHOBTB2_twohop': '9ae62403-0e5d-4be7-bdf6-fb7a2d1d358e',
 'A.9_EGFR_advanced': '463ba860-f94b-41a1-be95-779484d67522',
 'A.2b_expanded_RHOBTB2_twohop': '1b2a48de-24cb-417b-ad9e-5fb2250698da',
 'A.1_RHOBTB2': '27cf337e-5c0d-4ba2-9f80-866c72d57bab',
 'A.2a_expanded_RHOBTB2_twohop': '644033e1-c57f-492c-9bf6-3e8febc703d2',
 'B.5_DILI-one-hop-from-genes': '23e035b0-080d-4246-b621-de0b4388b0b3',
 'B.0_Initial_three-hop_MESH:D056487_ChronicDILI': '35b52c1a-50ed-4273-8f62-18384256d7d7',
 'B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin': '9c41c870-f6ca-4428-96a2-30237da58bd6',
 'B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin': '829711f7-ae7b-4e65-8faf-5c87f79653af',
 'B.3c_DIL

In [6]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 72418a31-e5fb-47e2-ad89-b7b96e35db55
Done
Done
ara-aragorn Done 280
Done
ara-arax Done 138
Error
ara-bte Error 0
A.2b_RHOBTB2_twohop 10d8a126-2785-4695-98f5-650f1a85fc85
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte ARS Error 0
A.2a_RHOBTB2_twohop 30daf606-6f8f-4d82-bd0f-9f21bd017552
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte ARS Error 0
A.8_EGFR_simple 3ab1c5de-810f-4b9b-88d0-dec261b1b6d6
Done
Done
ara-aragorn Done 1017
Done
ara-arax Done 433
Done
ara-bte ARS Error 0
A.0_RHOBTB2_direct 05aec4a6-9dee-4578-aaad-58858215300e
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Error
ara-bte Error 0
A.2_RHOBTB2_twohop 9ae62403-0e5d-4be7-bdf6-fb7a2d1d358e
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 361
Error
ara-bte Error 0
A.9_EGFR_advanced 463ba860-f94b-41a1-be95-779484d67522
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 170
Error
ara-bte Error 0
A.2b_expanded_RHOBTB2_twohop 1b2a48de-24cb-417b-ad9e-5fb2250698da
Done
Done
ara-arax Done 19

### Creating dataframe for workflows with PK

<br>

In [7]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2b_RHOBTB2_twohop
A.2b_expanded_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_three-hop_MESH:D056487_ChronicDILI
B.1a_DILI-three-hop-from-MONDO:0005359_DILI
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI
B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.2a_DILI-three-hop-from-MONDO:0005359_DILI
B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI
B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH:D00007738

In [8]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [9]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=05aec4a6-...,https://arax.ncats.io/?source=ARS&id=27cf337e-...,https://arax.ncats.io/?source=ARS&id=9ae62403-...,https://arax.ncats.io/?source=ARS&id=30daf606-...,https://arax.ncats.io/?source=ARS&id=644033e1-...,https://arax.ncats.io/?source=ARS&id=10d8a126-...,https://arax.ncats.io/?source=ARS&id=1b2a48de-...,https://arax.ncats.io/?source=ARS&id=72418a31-...,https://arax.ncats.io/?source=ARS&id=3ab1c5de-...,https://arax.ncats.io/?source=ARS&id=463ba860-...,...,https://arax.ncats.io/?source=ARS&id=23e035b0-...,https://arax.ncats.io/?source=ARS&id=6d1c8e9e-...,https://arax.ncats.io/?source=ARS&id=fed5f777-...,https://arax.ncats.io/?source=ARS&id=2d442663-...,https://arax.ncats.io/?source=ARS&id=0d5e7743-...,https://arax.ncats.io/?source=ARS&id=92945720-...,https://arax.ncats.io/?source=ARS&id=b256b662-...,https://arax.ncats.io/?source=ARS&id=4964617a-...,https://arax.ncats.io/?source=ARS&id=0cd21616-...,https://arax.ncats.io/?source=ARS&id=03492b17-...
ara-aragorn,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,...,Results: 200,Results: 200,No Results: 422,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200
ara-arax,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200
ara-bte,Error: 504,Error: 504,Error: 504,No Results: 200,Error: 504,No Results: 200,Error: 504,Error: 502,No Results: 200,Error: 504,...,Error: 504,Error: 504,No Results: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504


### Creating second table with edge attribute source

<br>

In [10]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2b_RHOBTB2_twohop
A.2b_expanded_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_three-hop_MESH:D056487_ChronicDILI
B.1a_DILI-three-hop-from-MONDO:0005359_DILI
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI
B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.2a_DILI-three-hop-from-MONDO:0005359_DILI
B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI
B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH:D00007738

In [11]:
final_dict2

defaultdict(dict,
            {'A.1_RHOBTB2': {'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-robokop',
               'infores:lincs',
               'infores:rtx-kg2',
               'infores:semmeddb',
               'infores:spoke'},
              'ara-arax': {'infores:arax',
               'infores:biothings-semmeddb-gene',
               'infores:rtx-kg2',
               'infores:semmeddb',
               'infores:translator-biothings-explorer'}},
             'A.2_RHOBTB2_twohop': {'ara-arax': {'infores:arax',
               'infores:connections-hypothesis',
               'infores:dgidb',
               'infores:drugbank',
               'infores:rtx-kg2',
               'infores:semmeddb',
               'infores:uniprot'}},
             'A.2a_expanded_RHOBTB2_twohop': {'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-robokop',
               'infores

In [12]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [13]:
final_dictassemble

[['A.1_RHOBTB2', 'ara-aragorn', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:automat-robokop'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:translator-biothings-explorer'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:biothings-semmeddb-gene'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:uniprot'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:connections-hypothesis'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:drugbank'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:rtx-kg2'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:arax'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:s

In [14]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [15]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [16]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [17]:
df2test = pd.DataFrame(df2test.unstack().T)

In [18]:
df2test

Workflow,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-MONDO:0005359_DILI,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
icees:dili,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,[ara-arax],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,NaN,NaN,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:aragorn-ranker-ara,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,NaN,NaN,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax]
infores:automat-robokop,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,NaN,NaN,...,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn]
infores:biothings-dgidb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:biothings-multiomics-biggim-drugresponse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:biothings-multiomics-clinical-risk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:biothings-semmeddb-gene,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#df2test.drop([''], axis=0, inplace=True)

In [20]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [21]:
#df2.replace([], 'None', regex=True,inplace=True)

In [22]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [23]:
df2test = df2test.rename_axis(None)

In [24]:
df2test.columns.name = None

In [25]:
df2test.fillna('', inplace=True)

In [26]:
df2test

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-MONDO:0005359_DILI,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.6_metformin-ferritin
icees:dili,,,,,,,,[ara-arax],,[ara-arax],...,,,,,,,,,,
infores:aragorn,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,,,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:aragorn-ranker-ara,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,,,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax]
infores:automat-robokop,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,,,...,[ara-aragorn],[ara-aragorn],,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],,[ara-aragorn]
infores:biothings-dgidb,,,,,,,,,,,...,,,[ara-arax],,,,,,,
infores:biothings-multiomics-biggim-drugresponse,,,,,,,,,,,...,,,[ara-arax],,,,,,,
infores:biothings-multiomics-clinical-risk,,,,,,,,,,,...,,,,,,,,,,
infores:biothings-semmeddb-gene,[ara-arax],,,,,,,,,,...,,,,,,,,,,
infores:cam-kp,,,,,,,,,,,...,,,,[ara-aragorn],,[ara-aragorn],,,,


In [27]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [28]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]
infores_catalog = infores_catalog[:335]
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

        
        
        
        
        
        
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]
df2test['Query Type'] = 'Async'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

In [29]:
df2test

,Query Type,Translator_Category_Complaint_to_ColL&M_InforesCatalog,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-MONDO:0005359_DILI,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.6_metformin-ferritin
infores:genetics-data-provider,Async,KP,,,,,,,,,...,,,,,,[ara-aragorn],"[ara-aragorn, ara-arax]",[ara-aragorn],,
infores:text-mining-provider-targeted,Async,KP,,,,,,,,,...,,,[ara-arax],,,,,,,
infores:automat-robokop,Async,KP,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,...,[ara-aragorn],[ara-aragorn],,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],,[ara-aragorn]
infores:biothings-dgidb,Async,KP,,,,,,,,,...,,,[ara-arax],,,,,,,
infores:biothings-multiomics-biggim-drugresponse,Async,KP,,,,,,,,,...,,,[ara-arax],,,,,,,
infores:biothings-multiomics-clinical-risk,Async,KP,,,,,,,,,...,,,,,,,,,,
infores:biothings-semmeddb-gene,Async,KP,[ara-arax],,,,,,,,...,,,,,,,,,,
infores:cam-kp,Async,KP,,,,,,,,,...,,,,[ara-aragorn],,[ara-aragorn],,,,
infores:spoke,Async,KP,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,,,,...,,,,,,,,,,[ara-aragorn]
infores:connections-hypothesis,Async,KP,,[ara-arax],,,,,,,...,,,,,,,,,,


<br>

## Converting the Pk's to hyperlink

<br>

In [30]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [31]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [32]:
df.sort_index(inplace=True)

In [33]:
df['Query Type'] = 'Async'

df = df[['Query Type']+ list(df.columns[:-1])]

In [34]:
df

,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,...,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Async,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 422,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200
ara-arax,Async,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200
ara-bte,Async,Error: 504,Error: 504,Error: 504,No Results: 200,Error: 504,No Results: 200,Error: 504,Error: 502,No Results: 200,...,Error: 504,Error: 504,No Results: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
pk,Async,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...",...,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&..."


In [35]:
#date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
#wks_name = 'Workflow Progress Tracker Asynchronous_' + date

In [36]:
#wks_name

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [37]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
import gspread_dataframe as gd

In [38]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [39]:
ws = gc.open("workflow_progress_tracker").worksheet("Workflow Progress Tracker_2021_09_21-07_16_26_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

existing.loc['pk'][:] = existing.loc['pk'][:].apply(lambda x: x.replace('" "', '","'))

existing.loc['pk'][1]

updated = existing.append(df)
gd.set_with_dataframe(ws, updated,include_index=True,include_column_header=True)

In [40]:
#spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
#d2g.upload(df, spreadsheet_key, "TESTING ONLY", credentials=credentials, row_names=True)

In [41]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
#sh = wksh.worksheet(wks_name)
sh = wksh.worksheet("Workflow Progress Tracker_2021_09_21-07_16_26_AM")

In [42]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [43]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1423660451,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [44]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1423660451,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1423660451,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'r

In [45]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1423660451,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1423660451,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bo

In [46]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1423660451,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1423660451,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
    

In [47]:
sh

<Worksheet 'Workflow Progress Tracker_2021_09_21-07_16_26_AM' id:1423660451>

In [48]:
# Select a range
cell_list = sh.range('B19:AO19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_21-07_16_26_AM'!B19:AO19",
 'updatedRows': 1,
 'updatedColumns': 40,
 'updatedCells': 40}

In [49]:
# Select a range
cell_list = sh.range('B23:AO23')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_21-07_16_26_AM'!B23:AO23",
 'updatedRows': 1,
 'updatedColumns': 40,
 'updatedCells': 40}

### Push Dataframe 2

<br>

In [50]:
ws2 = gc.open("workflow_progress_tracker").worksheet("edge_attribute_source_2021_09_21-07_16_26_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws2.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

updated = existing.append(df2test)
gd.set_with_dataframe(ws2, updated,include_index=True,include_column_header=True)
